In [15]:
import torch
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from PIL import Image
import torch.nn as nn
import torch.optim as optim
import random

In [16]:
class_names = [
    'calling',
    'clapping',
    'cycling',
    'dancing',
    'drinking',
    'eating',
    'fightning',
    'hugging',
    'laughing',
    'listening_to_music',
    'running',
    'sitting',
    'sleeping',
    'texting',
    'using_laptop'
]

# classify an image with the given model path
def classify_image(batch_data, model, device):
    image_list = []

    # Need to test on batch size of 36 to match training size
    for idx, row in batch_data.iterrows():
      image_path = f"/Users/clintlee/Development/vscode/CS685/data2/test/{row[1]}/{row[0]}"

      transform = transforms.Compose([
          transforms.Resize((128, 128)),
          transforms.ToTensor(),
          transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
      ])
      
      image = Image.open(image_path).convert('RGB')
      image = transform(image).unsqueeze(0).to(device)

      image_list.append(image)

    image_tensor = torch.cat(image_list, dim=0)

    model.eval()

    with torch.no_grad():
        outputs = model(image_tensor)
        _, predicted = torch.max(outputs, 1)

    return predicted

In [17]:
import pandas as pd
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the model
model = models.convnext_base(weights='DEFAULT').to(device)
    
in_features = model.classifier[2].in_features
num_classes = 15

# Modify classifier
model.classifier = nn.Sequential(
    nn.LayerNorm(( 36,1024,1,1)), 
    nn.Flatten(1),
    nn.Linear(in_features, num_classes)
).to(device)

model.load_state_dict(torch.load("/Users/clintlee/Development/vscode/CS685/models/convnext/convnext_model.pth", map_location=torch.device("cpu")))


# Read the CSV file
# Using absolute path similar to the model path
test_data = pd.read_csv("/Users/clintlee/Development/vscode/CS685/data2/test/test_labels.csv")
print(f"Test dataset contains {len(test_data)} images")

# Assuming first column is filename and third is the true label
# Adjust these if your CSV has different structure
filename_col = 0
folder_col = 1
label_col = 2

# Process images and compute accuracy
correct = 0
total = 0

num_batches = len(test_data) / 36


# Process images in batches of 36
batch_size = 36
num_complete_batches = int(len(test_data) // batch_size)

for batch_idx in range(num_complete_batches):
    try:
        # Get corresponding true labels from test_data
        start_idx = batch_idx * batch_size
        end_idx = (batch_idx + 1) * batch_size
        batch_data = test_data.iloc[start_idx:end_idx]
        true_labels = batch_data.iloc[:, label_col].tolist()

        # Get predictions for a batch of 36 images
        predicted_labels = classify_image(batch_data, model, device)
        
        # Compare predictions with true labels
        for idx, (pred, true) in enumerate(zip(predicted_labels, true_labels)):
            if pred.item() == true:
                correct += 1
            total += 1
        
        # Print progress update
        print(f"Processed batch {batch_idx+1}/{num_complete_batches} ({end_idx}/{len(test_data)} images)")
    
    except Exception as e:
        print(f"Error processing batch {batch_idx+1}: {e}")

# Handle remaining images if any (less than a batch)
remaining = len(test_data) % batch_size
if remaining > 0:
    print(f"Note: {remaining} images remain unprocessed as they don't form a complete batch of 36")

# Calculate accuracy
accuracy = 100 * correct / total
print(f"\nAccuracy: {accuracy:.2f}% ({correct}/{total})")

Test dataset contains 1890 images


/var/folders/77/yjvjk5s92_gc6p2sqr10794w0000gn/T/ipykernel_86336/1017574126.py:25: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  image_path = f"/Users/clintlee/Development/vscode/CS685/data2/test/{row[1]}/{row[0]}"


Processed batch 1/52 (36/1890 images)
Processed batch 2/52 (72/1890 images)
Processed batch 3/52 (108/1890 images)
Processed batch 4/52 (144/1890 images)
Processed batch 5/52 (180/1890 images)
Processed batch 6/52 (216/1890 images)
Processed batch 7/52 (252/1890 images)
Processed batch 8/52 (288/1890 images)
Processed batch 9/52 (324/1890 images)
Processed batch 10/52 (360/1890 images)
Processed batch 11/52 (396/1890 images)
Processed batch 12/52 (432/1890 images)
Processed batch 13/52 (468/1890 images)
Processed batch 14/52 (504/1890 images)
Processed batch 15/52 (540/1890 images)
Processed batch 16/52 (576/1890 images)
Processed batch 17/52 (612/1890 images)
Processed batch 18/52 (648/1890 images)
Processed batch 19/52 (684/1890 images)
Processed batch 20/52 (720/1890 images)
Processed batch 21/52 (756/1890 images)
Processed batch 22/52 (792/1890 images)
Processed batch 23/52 (828/1890 images)
Processed batch 24/52 (864/1890 images)
Processed batch 25/52 (900/1890 images)
Processed b